In [1]:
import plotly.express as pe
from plotly.figure_factory import create_distplot
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import numpy as np

In [2]:
#Create df from hatchery file
hatchery = "WALLACE R HATCHERY"
species = "Chinook"
esc = pd.read_csv('wdfwHatchery6_18.csv', low_memory=False)
esc['Date'] = pd.to_datetime(esc['Date'])
esc = esc[['Date', 'Facility', 'Adult Count', 'Species', 'Event']]
esc = esc[esc.Facility == hatchery]
esc = esc[esc.Event == 'Trap Estimate']
esc = esc[esc.Species == species]
esc['DOY'] = esc['Date'].dt.strftime('%j')
esc = esc.sort_values(by="Date")
esc = esc[esc['Adult Count'] != 0]


esc.head(180)

,Date,Facility,Adult Count,Species,Event,DOY
170544,1997-07-03,WALLACE R HATCHERY,63,Chinook,Trap Estimate,184
196880,1997-07-07,WALLACE R HATCHERY,119,Chinook,Trap Estimate,188
234933,1997-07-10,WALLACE R HATCHERY,42,Chinook,Trap Estimate,191
199688,1997-07-22,WALLACE R HATCHERY,142,Chinook,Trap Estimate,203
208434,1997-07-23,WALLACE R HATCHERY,36,Chinook,Trap Estimate,204
...,...,...,...,...,...,...
174369,2006-09-20,WALLACE R HATCHERY,22,Chinook,Trap Estimate,263
182747,2006-09-20,WALLACE R HATCHERY,501,Chinook,Trap Estimate,263
183905,2006-09-22,WALLACE R HATCHERY,25,Chinook,Trap Estimate,265
233513,2006-09-29,WALLACE R HATCHERY,12,Chinook,Trap Estimate,272


In [3]:
df_time = esc
df_time = df_time.sort_values(by="DOY")
first = df_time.iloc[0].DOY
last = df_time.iloc[-1].DOY
print(first, last)

138 316


In [4]:
esc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1139 entries, 170544 to 439019
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         1139 non-null   datetime64[ns]
 1   Facility     1139 non-null   object        
 2   Adult Count  1139 non-null   int64         
 3   Species      1139 non-null   object        
 4   Event        1139 non-null   object        
 5   DOY          1139 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 62.3+ KB


In [5]:
esc['DOY'] = esc['DOY'].astype(np.int64)
esc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1139 entries, 170544 to 439019
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         1139 non-null   datetime64[ns]
 1   Facility     1139 non-null   object        
 2   Adult Count  1139 non-null   int64         
 3   Species      1139 non-null   object        
 4   Event        1139 non-null   object        
 5   DOY          1139 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 62.3+ KB


In [6]:
#Creat df from rivers file
river = 'sky.txt'
df = pd.read_csv(river, sep='\t', header=None)
df = df.rename(columns ={2:"Date",3:"cfs"})
df = df.drop(columns=[0,1,4],)
df['Date'] = df['Date'].astype('datetime64[ns]')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8744 entries, 0 to 8743
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    8744 non-null   datetime64[ns]
 1   cfs     8744 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 136.8 KB


In [7]:
#Merge data frames and trim for species date range
plt_tst = pd.merge(df, esc, on=['Date'], how='outer')
plt_tst['rolling14'] = plt_tst.cfs.rolling(14).mean()
plt_tst['rolling7'] = plt_tst.cfs.rolling(7).mean()
plt_tst['rolling3'] = plt_tst.cfs.rolling(3).mean()


plt_tst.head(180)

,Date,cfs,Facility,Adult Count,Species,Event,DOY,rolling14,rolling7,rolling3
0,1997-07-03,6380.0,WALLACE R HATCHERY,63.0,Chinook,Trap Estimate,184.0,NaN,NaN,NaN
1,1997-07-04,7480.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997-07-05,8490.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7450.000000
3,1997-07-06,9120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8363.333333
4,1997-07-07,7690.0,WALLACE R HATCHERY,119.0,Chinook,Trap Estimate,188.0,NaN,NaN,8433.333333
...,...,...,...,...,...,...,...,...,...,...
175,1997-12-25,2080.0,NaN,NaN,NaN,NaN,NaN,3766.428571,3025.714286,2300.000000
176,1997-12-26,1970.0,NaN,NaN,NaN,NaN,NaN,3779.285714,2672.857143,2110.000000
177,1997-12-27,2130.0,NaN,NaN,NaN,NaN,NaN,3810.000000,2450.000000,2060.000000
178,1997-12-28,4160.0,NaN,NaN,NaN,NaN,NaN,3990.000000,2570.000000,2753.333333


In [8]:
rt = (plt_tst['DOY'] >= first) & (plt_tst['DOY'] <= last)
plt_tst = plt_tst.loc[rt]
plt_tst = plt_tst.sort_values(by="DOY")

TypeError: Invalid comparison between dtype=float64 and str

In [ ]:
# fig = sns.displot(esc, x="DOY", kind='kde', weights='Adult Count')

In [ ]:
def logReg(df_in):
    feature_cols = ['cfs','DOY'] #independent variables
    X = df_in[feature_cols] 
    y = df_in.returnBinary #target variable
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)
    
    logreg = LogisticRegression()
    
    # fit the model with data
    logreg.fit(X_train,y_train)

    y_pred=logreg.predict(X_test)
    
    cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
    print(cnf_matrix)
    

    
    score = logreg.score(X_test, y_test)
    print(score)
    
    odds = np.exp(logreg.coef_[0])
    stats = pd.DataFrame(odds,
            X.columns,
            columns=['coef'])
    rstats = pd.DataFrame(logreg.coef_[0],
            X.columns,
            columns=['coef'])
    return(stats, rstats)

In [ ]:
logReg()